<a href="https://colab.research.google.com/github/Sikander-creater/Examination-of-CSC-programming/blob/main/Simple_Personal_Budget_Manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyinputplus


In [17]:
import pandas as pd
from pathlib import Path
import pyinputplus as pyip

CSV_PATH = Path('budget.csv')
COLUMNS = ['date','description','type','amount']


In [18]:
import pandas as pd

def load_data(path: Path = CSV_PATH) -> pd.DataFrame:
    try:
        df = pd.read_csv(path)
        missing = [c for c in COLUMNS if c not in df.columns]
        if missing:
            raise ValueError(f"CSV missing columns: {missing}")
        df['amount'] = pd.to_numeric(df['amount'], errors='coerce').fillna(0.0)
        df['type'] = df['type'].astype(str).str.lower()
        df['date'] = df['date'].astype(str)
        df['description'] = df['description'].astype(str)
        return df[COLUMNS]
    except FileNotFoundError:
        print("No existing budget.csv found. Starting a new ledger.")
        return pd.DataFrame(columns=COLUMNS)
    except Exception as e:
        print(f"Warning: Problem reading CSV ({e}). Starting a new empty ledger.")
        return pd.DataFrame(columns=COLUMNS)

def save_data(df: pd.DataFrame, path: Path = CSV_PATH) -> None:
    df.to_csv(path, index=False)

def calculate_balance(df: pd.DataFrame) -> float:
    if df.empty:
        return 0.0
    inc = df.loc[df['type'] == 'income', 'amount'].sum()
    exp = df.loc[df['type'] == 'expense', 'amount'].sum()
    return float(inc - exp)

def add_transaction(df: pd.DataFrame) -> pd.DataFrame:
    date_val = pyip.inputDate(prompt="Date (YYYY-MM-DD): ", formats=['%Y-%m-%d'])
    desc_val = pyip.inputStr("Description: ", min=1)
    type_val = pyip.inputChoice(["income","expense"], prompt="Type (income/expense): ")
    amt_val = pyip.inputFloat("Amount (>=0.01): ", min=0.01)
    row = {
        'date': date_val.strftime('%Y-%m-%d'),
        'description': desc_val,
        'type': type_val.lower(),
        'amount': float(amt_val)
    }
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    print("✔ Transaction added.")
    return df

def view_transactions(df: pd.DataFrame) -> None:
    if df.empty:
        print("No transactions yet.")
    else:
        print(df.to_string(index=False))

def show_menu() -> int:
    print("\n== Simple Personal Budget Manager ==")
    print("1) Add Transaction")
    print("2) View All Transactions")
    print("3) View Current Balance")
    print("4) Save & Exit")
    return pyip.inputInt("Choose 1-4: ", min=1, max=4)

def main():
    df = load_data()
    while True:
        choice = show_menu()
        if choice == 1:
            df = add_transaction(df)
        elif choice == 2:
            view_transactions(df)
        elif choice == 3:
            bal = calculate_balance(df)
            print(f"Current Balance: ${bal:,.2f}")
        elif choice == 4:
            save_data(df)
            print("Goodbye!")
            break


In [10]:
test_df = pd.DataFrame([
    {'date':'2025-01-01','description':'gift','type':'income','amount':100.0},
    {'date':'2025-01-02','description':'shift pay','type':'income','amount':350.0},
    {'date':'2025-01-03','description':'groceries','type':'expense','amount':120.50},
], columns=COLUMNS)

assert abs(calculate_balance(test_df) - 329.50) < 1e-9
assert calculate_balance(pd.DataFrame(columns=COLUMNS)) == 0.0
print('All asserts passed ✅')


All asserts passed ✅


In [ ]:
main()


No existing budget.csv found. Starting a new ledger.

== Simple Personal Budget Manager ==
1) Add Transaction
2) View All Transactions
3) View Current Balance
4) Save & Exit
Choose 1-4: 